# Inflation Metrics  
  
## CPI and PPI  
Website location:  
https://www.bls.gov/data/#prices

In [4]:
import pandas as pd
# import matplotlib.dates as mdates
# import matplotlib.pyplot as plt
# from datetime import datetime
from openpyxl import Workbook
import hvplot.pandas
# import numpy as np
# from functools import reduce
# import yfinance as yf # https://pypi.org/project/yfinance/

In [5]:
# Read data
# CPI data
cpi_df = pd.read_excel('./Inputs/BLS_data/CPI_for_All_Urban_Consumers.xlsx', engine='openpyxl', header=10)
header_row = 0
cpi_df.columns = cpi_df.iloc[header_row]
cpi_df = cpi_df.drop(header_row)
cpi_df = cpi_df.reset_index(drop=True)
del cpi_df['HALF1']
del cpi_df['HALF2']
cpi_df['Year'] = cpi_df['Year'].astype(int)
months = cpi_df.columns.tolist()
months.pop(0)
years = cpi_df['Year'].to_list()

# Build year and month dataframe
date = []
for year in years:
    for month in months:
        x = (f"{year}-{month}")
        date.append(x)

del cpi_df['Year']

# Build CPI values list
cpi_values = []
x = []
for i in range(len(cpi_df.index)):
    x = cpi_df.values.tolist()[i]
    cpi_values = cpi_values + x

date_series = pd.DataFrame([date])
date_series = date_series.T
date_series = date_series.rename(columns={date_series.columns[0]:"Date"})
cpi_val_series = pd.DataFrame([cpi_values])
cpi_val_series = cpi_val_series.T
cpi_val_series = cpi_val_series.rename(columns={cpi_val_series.columns[0]:"CPI"})
cpi_df2 = pd.concat([date_series, cpi_val_series], axis=1).reindex(date_series.index)
cpi_df2 = cpi_df2.dropna()
cpi_df2 = cpi_df2.set_index(['Date'])
cpi_df2['CPI'] = cpi_df2['CPI'].astype(float)
cpi_df2.index = pd.to_datetime(cpi_df2.index)
cpi_df2['CPI_YoY_Pct_Chg'] = (cpi_df2['CPI'] - cpi_df2['CPI'].shift(12)) / cpi_df2['CPI'].shift(12) * 100
cpi_df2['CPI_YoY_Pct_Chg'] = cpi_df2['CPI_YoY_Pct_Chg'].round(2)
cpi_df2['CPI_All'] = (cpi_df2['CPI_YoY_Pct_Chg'] - cpi_df2['CPI_YoY_Pct_Chg'].mean()) / cpi_df2['CPI_YoY_Pct_Chg'].std()
cpi_df2['CPI_All'] = cpi_df2['CPI_All'].round(2)
cpi_all_df = cpi_df2[['CPI_All']]
cpi_all_df

,CPI_All
Date,
1913-01-01,NaN
1913-02-01,NaN
1913-03-01,NaN
1913-04-01,NaN
1913-05-01,NaN
...,...
2021-03-01,-0.12
2021-04-01,0.19
2021-05-01,0.36


In [23]:
# Read data
# PPI data
cpi_df = pd.read_excel('./Inputs/BLS_data/PPI_for_All_Commodities.xlsx', engine='openpyxl', header=10)
header_row = 0
cpi_df.columns = cpi_df.iloc[header_row]
cpi_df = cpi_df.drop(header_row)
cpi_df = cpi_df.reset_index(drop=True)
# del cpi_df['HALF1']
# del cpi_df['HALF2']
cpi_df['Year'] = cpi_df['Year'].astype(int)
months = cpi_df.columns.tolist()
months.pop(0)
years = cpi_df['Year'].to_list()

# Build year and month dataframe
date = []
for year in years:
    for month in months:
        x = (f"{year}-{month}")
        date.append(x)

del cpi_df['Year']

# Build CPI values list
cpi_values = []
x = []
for i in range(len(cpi_df.index)):
    x = cpi_df.values.tolist()[i]
    cpi_values = cpi_values + x

date_series = pd.DataFrame([date])
date_series = date_series.T
date_series = date_series.rename(columns={date_series.columns[0]:"Date"})
cpi_val_series = pd.DataFrame([cpi_values])
cpi_val_series = cpi_val_series.T
cpi_val_series = cpi_val_series.rename(columns={cpi_val_series.columns[0]:"CPI"})
cpi_df2 = pd.concat([date_series, cpi_val_series], axis=1).reindex(date_series.index)
cpi_df2 = cpi_df2.dropna()
cpi_df2 = cpi_df2.set_index(['Date'])
cpi_df2['CPI'] = cpi_df2['CPI'].astype(float)
cpi_df2.index = pd.to_datetime(cpi_df2.index)
cpi_df2['CPI_YoY_Pct_Chg'] = (cpi_df2['CPI'] - cpi_df2['CPI'].shift(12)) / cpi_df2['CPI'].shift(12) * 100
cpi_df2['CPI_YoY_Pct_Chg'] = cpi_df2['CPI_YoY_Pct_Chg'].round(2)
cpi_df2['PPI_All'] = (cpi_df2['CPI_YoY_Pct_Chg'] - cpi_df2['CPI_YoY_Pct_Chg'].mean()) / cpi_df2['CPI_YoY_Pct_Chg'].std()
cpi_df2['PPI_All'] = cpi_df2['PPI_All'].round(2)
ppi_all_df = cpi_df2[['PPI_All']]
ppi_all_df

,PPI_All
Date,
1913-01-01,NaN
1913-02-01,NaN
1913-03-01,NaN
1913-04-01,NaN
1913-05-01,NaN
...,...
2021-04-01,1.64
2021-05-01,1.81
2021-06-01,1.87


In [24]:
cpi_ppi_df = cpi_all_df.merge(ppi_all_df, how='outer', left_index=True, right_index=True)

cpi_ppi_plot = cpi_ppi_df.hvplot.line(title='CPI All',
                                                grid=True,height=500,width=800)
cpi_ppi_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [7]:
# Read data
# CPI data
cpi_df = pd.read_excel('./Inputs/BLS_data/CPI_for_All_Urban_Consumers_Food.xlsx', engine='openpyxl', header=10)
header_row = 0
cpi_df.columns = cpi_df.iloc[header_row]
cpi_df = cpi_df.drop(header_row)
cpi_df = cpi_df.reset_index(drop=True)
del cpi_df['HALF1']
del cpi_df['HALF2']
cpi_df['Year'] = cpi_df['Year'].astype(int)
months = cpi_df.columns.tolist()
months.pop(0)
years = cpi_df['Year'].to_list()

# Build year and month dataframe
date = []
for year in years:
    for month in months:
        x = (f"{year}-{month}")
        date.append(x)

del cpi_df['Year']

# Build CPI values list
cpi_values = []
x = []
for i in range(len(cpi_df.index)):
    x = cpi_df.values.tolist()[i]
    cpi_values = cpi_values + x

date_series = pd.DataFrame([date])
date_series = date_series.T
date_series = date_series.rename(columns={date_series.columns[0]:"Date"})
cpi_val_series = pd.DataFrame([cpi_values])
cpi_val_series = cpi_val_series.T
cpi_val_series = cpi_val_series.rename(columns={cpi_val_series.columns[0]:"CPI"})
cpi_df2 = pd.concat([date_series, cpi_val_series], axis=1).reindex(date_series.index)
cpi_df2 = cpi_df2.dropna()
cpi_df2 = cpi_df2.set_index(['Date'])
cpi_df2['CPI'] = cpi_df2['CPI'].astype(float)
cpi_df2.index = pd.to_datetime(cpi_df2.index)
cpi_df2['CPI_YoY_Pct_Chg'] = (cpi_df2['CPI'] - cpi_df2['CPI'].shift(12)) / cpi_df2['CPI'].shift(12) * 100
cpi_df2['CPI_YoY_Pct_Chg'] = cpi_df2['CPI_YoY_Pct_Chg'].round(2)
cpi_df2['CPI_Food'] = (cpi_df2['CPI_YoY_Pct_Chg'] - cpi_df2['CPI_YoY_Pct_Chg'].mean()) / cpi_df2['CPI_YoY_Pct_Chg'].std()
cpi_df2['CPI_Food'] = cpi_df2['CPI_Food'].round(2)
cpi_food_df = cpi_df2[['CPI_Food']]
cpi_food_df

,CPI_Food
Date,
1913-01-01,NaN
1913-02-01,NaN
1913-03-01,NaN
1913-04-01,NaN
1913-05-01,NaN
...,...
2021-03-01,0.01
2021-04-01,-0.14
2021-05-01,-0.17


In [8]:
# Read data
# CPI data
cpi_df = pd.read_excel('./Inputs/BLS_data/CPI_for_All_Urban_Consumers_Housing.xlsx', engine='openpyxl', header=10)
header_row = 0
cpi_df.columns = cpi_df.iloc[header_row]
cpi_df = cpi_df.drop(header_row)
cpi_df = cpi_df.reset_index(drop=True)
del cpi_df['HALF1']
del cpi_df['HALF2']
cpi_df['Year'] = cpi_df['Year'].astype(int)
months = cpi_df.columns.tolist()
months.pop(0)
years = cpi_df['Year'].to_list()

# Build year and month dataframe
date = []
for year in years:
    for month in months:
        x = (f"{year}-{month}")
        date.append(x)

del cpi_df['Year']

# Build CPI values list
cpi_values = []
x = []
for i in range(len(cpi_df.index)):
    x = cpi_df.values.tolist()[i]
    cpi_values = cpi_values + x

date_series = pd.DataFrame([date])
date_series = date_series.T
date_series = date_series.rename(columns={date_series.columns[0]:"Date"})
cpi_val_series = pd.DataFrame([cpi_values])
cpi_val_series = cpi_val_series.T
cpi_val_series = cpi_val_series.rename(columns={cpi_val_series.columns[0]:"CPI"})
cpi_df2 = pd.concat([date_series, cpi_val_series], axis=1).reindex(date_series.index)
cpi_df2 = cpi_df2.dropna()
cpi_df2 = cpi_df2.set_index(['Date'])
cpi_df2['CPI'] = cpi_df2['CPI'].astype(float)
cpi_df2.index = pd.to_datetime(cpi_df2.index)
cpi_df2['CPI_YoY_Pct_Chg'] = (cpi_df2['CPI'] - cpi_df2['CPI'].shift(12)) / cpi_df2['CPI'].shift(12) * 100
cpi_df2['CPI_YoY_Pct_Chg'] = cpi_df2['CPI_YoY_Pct_Chg'].round(2)
cpi_df2['CPI_Housing'] = (cpi_df2['CPI_YoY_Pct_Chg'] - cpi_df2['CPI_YoY_Pct_Chg'].mean()) / cpi_df2['CPI_YoY_Pct_Chg'].std()
cpi_df2['CPI_Housing'] = cpi_df2['CPI_Housing'].round(2)
cpi_housing_df = cpi_df2[['CPI_Housing']]
cpi_housing_df

,CPI_Housing
Date,
1967-01-01,NaN
1967-02-01,NaN
1967-03-01,NaN
1967-04-01,NaN
1967-05-01,NaN
...,...
2021-03-01,-0.66
2021-04-01,-0.51
2021-05-01,-0.42


In [9]:
# Read data
# CPI data
cpi_df = pd.read_excel('./Inputs/BLS_data/CPI_for_All_Urban_Consumers_Medical_Care.xlsx', engine='openpyxl', header=10)
header_row = 0
cpi_df.columns = cpi_df.iloc[header_row]
cpi_df = cpi_df.drop(header_row)
cpi_df = cpi_df.reset_index(drop=True)
del cpi_df['HALF1']
del cpi_df['HALF2']
cpi_df['Year'] = cpi_df['Year'].astype(int)
months = cpi_df.columns.tolist()
months.pop(0)
years = cpi_df['Year'].to_list()

# Build year and month dataframe
date = []
for year in years:
    for month in months:
        x = (f"{year}-{month}")
        date.append(x)

del cpi_df['Year']

# Build CPI values list
cpi_values = []
x = []
for i in range(len(cpi_df.index)):
    x = cpi_df.values.tolist()[i]
    cpi_values = cpi_values + x

date_series = pd.DataFrame([date])
date_series = date_series.T
date_series = date_series.rename(columns={date_series.columns[0]:"Date"})
cpi_val_series = pd.DataFrame([cpi_values])
cpi_val_series = cpi_val_series.T
cpi_val_series = cpi_val_series.rename(columns={cpi_val_series.columns[0]:"CPI"})
cpi_df2 = pd.concat([date_series, cpi_val_series], axis=1).reindex(date_series.index)
cpi_df2 = cpi_df2.dropna()
cpi_df2 = cpi_df2.set_index(['Date'])
cpi_df2['CPI'] = cpi_df2['CPI'].astype(float)
cpi_df2.index = pd.to_datetime(cpi_df2.index)
cpi_df2['CPI_YoY_Pct_Chg'] = (cpi_df2['CPI'] - cpi_df2['CPI'].shift(12)) / cpi_df2['CPI'].shift(12) * 100
cpi_df2['CPI_YoY_Pct_Chg'] = cpi_df2['CPI_YoY_Pct_Chg'].round(2)
cpi_df2['CPI_Medical'] = (cpi_df2['CPI_YoY_Pct_Chg'] - cpi_df2['CPI_YoY_Pct_Chg'].mean()) / cpi_df2['CPI_YoY_Pct_Chg'].std()
cpi_df2['CPI_Medical'] = cpi_df2['CPI_Medical'].round(2)
cpi_medical_df = cpi_df2[['CPI_Medical']]
cpi_medical_df

,CPI_Medical
Date,
1935-03-01,NaN
1935-07-01,NaN
1935-10-01,NaN
1936-01-01,NaN
1936-04-01,NaN
...,...
2021-03-01,-1.20
2021-04-01,-1.31
2021-05-01,-1.50


In [10]:
# Read data
# CPI data
cpi_df = pd.read_excel('./Inputs/BLS_data/CPI_for_All_Urban_Consumers_Education_and_Communication.xlsx', engine='openpyxl', header=10)
header_row = 0
cpi_df.columns = cpi_df.iloc[header_row]
cpi_df = cpi_df.drop(header_row)
cpi_df = cpi_df.reset_index(drop=True)
del cpi_df['HALF1']
del cpi_df['HALF2']
cpi_df['Year'] = cpi_df['Year'].astype(int)
months = cpi_df.columns.tolist()
months.pop(0)
years = cpi_df['Year'].to_list()

# Build year and month dataframe
date = []
for year in years:
    for month in months:
        x = (f"{year}-{month}")
        date.append(x)

del cpi_df['Year']

# Build CPI values list
cpi_values = []
x = []
for i in range(len(cpi_df.index)):
    x = cpi_df.values.tolist()[i]
    cpi_values = cpi_values + x

date_series = pd.DataFrame([date])
date_series = date_series.T
date_series = date_series.rename(columns={date_series.columns[0]:"Date"})
cpi_val_series = pd.DataFrame([cpi_values])
cpi_val_series = cpi_val_series.T
cpi_val_series = cpi_val_series.rename(columns={cpi_val_series.columns[0]:"CPI"})
cpi_df2 = pd.concat([date_series, cpi_val_series], axis=1).reindex(date_series.index)
cpi_df2 = cpi_df2.dropna()
cpi_df2 = cpi_df2.set_index(['Date'])
cpi_df2['CPI'] = cpi_df2['CPI'].astype(float)
cpi_df2.index = pd.to_datetime(cpi_df2.index)
cpi_df2['CPI_YoY_Pct_Chg'] = (cpi_df2['CPI'] - cpi_df2['CPI'].shift(12)) / cpi_df2['CPI'].shift(12) * 100
cpi_df2['CPI_YoY_Pct_Chg'] = cpi_df2['CPI_YoY_Pct_Chg'].round(2)
cpi_df2['CPI_Education'] = (cpi_df2['CPI_YoY_Pct_Chg'] - cpi_df2['CPI_YoY_Pct_Chg'].mean()) / cpi_df2['CPI_YoY_Pct_Chg'].std()
cpi_df2['CPI_Education'] = cpi_df2['CPI_Education'].round(2)
cpi_education_df = cpi_df2[['CPI_Education']]
cpi_education_df

,CPI_Education
Date,
1993-01-01,NaN
1993-02-01,NaN
1993-03-01,NaN
1993-04-01,NaN
1993-05-01,NaN
...,...
2021-03-01,-0.27
2021-04-01,-0.12
2021-05-01,0.04


In [11]:
# Read data
# CPI data
cpi_df = pd.read_excel('./Inputs/BLS_data/CPI_for_All_Urban_Consumers_Motor_Fuel.xlsx', engine='openpyxl', header=10)
header_row = 0
cpi_df.columns = cpi_df.iloc[header_row]
cpi_df = cpi_df.drop(header_row)
cpi_df = cpi_df.reset_index(drop=True)
del cpi_df['HALF1']
del cpi_df['HALF2']
cpi_df['Year'] = cpi_df['Year'].astype(int)
months = cpi_df.columns.tolist()
months.pop(0)
years = cpi_df['Year'].to_list()

# Build year and month dataframe
date = []
for year in years:
    for month in months:
        x = (f"{year}-{month}")
        date.append(x)

del cpi_df['Year']

# Build CPI values list
cpi_values = []
x = []
for i in range(len(cpi_df.index)):
    x = cpi_df.values.tolist()[i]
    cpi_values = cpi_values + x

date_series = pd.DataFrame([date])
date_series = date_series.T
date_series = date_series.rename(columns={date_series.columns[0]:"Date"})
cpi_val_series = pd.DataFrame([cpi_values])
cpi_val_series = cpi_val_series.T
cpi_val_series = cpi_val_series.rename(columns={cpi_val_series.columns[0]:"CPI"})
cpi_df2 = pd.concat([date_series, cpi_val_series], axis=1).reindex(date_series.index)
cpi_df2 = cpi_df2.dropna()
cpi_df2 = cpi_df2.set_index(['Date'])
cpi_df2['CPI'] = cpi_df2['CPI'].astype(float)
cpi_df2.index = pd.to_datetime(cpi_df2.index)
cpi_df2['CPI_YoY_Pct_Chg'] = (cpi_df2['CPI'] - cpi_df2['CPI'].shift(12)) / cpi_df2['CPI'].shift(12) * 100
cpi_df2['CPI_YoY_Pct_Chg'] = cpi_df2['CPI_YoY_Pct_Chg'].round(2)
cpi_df2['CPI_Motor_Fuel'] = (cpi_df2['CPI_YoY_Pct_Chg'] - cpi_df2['CPI_YoY_Pct_Chg'].mean()) / cpi_df2['CPI_YoY_Pct_Chg'].std()
cpi_df2['CPI_Motor_Fuel'] = cpi_df2['CPI_Motor_Fuel'].round(2)
cpi_fuel_df = cpi_df2[['CPI_Motor_Fuel']]
cpi_fuel_df

,CPI_Motor_Fuel
Date,
1935-03-01,NaN
1935-07-01,NaN
1935-10-01,NaN
1936-01-01,NaN
1936-04-01,NaN
...,...
2021-03-01,1.12
2021-04-01,2.87
2021-05-01,3.30


In [12]:
# Read data
# CPI data
cpi_df = pd.read_excel('./Inputs/BLS_data/CPI_for_All_Urban_Consumers_New_Vehicles.xlsx', engine='openpyxl', header=10)
header_row = 0
cpi_df.columns = cpi_df.iloc[header_row]
cpi_df = cpi_df.drop(header_row)
cpi_df = cpi_df.reset_index(drop=True)
del cpi_df['HALF1']
del cpi_df['HALF2']
cpi_df['Year'] = cpi_df['Year'].astype(int)
months = cpi_df.columns.tolist()
months.pop(0)
years = cpi_df['Year'].to_list()

# Build year and month dataframe
date = []
for year in years:
    for month in months:
        x = (f"{year}-{month}")
        date.append(x)

del cpi_df['Year']

# Build CPI values list
cpi_values = []
x = []
for i in range(len(cpi_df.index)):
    x = cpi_df.values.tolist()[i]
    cpi_values = cpi_values + x

date_series = pd.DataFrame([date])
date_series = date_series.T
date_series = date_series.rename(columns={date_series.columns[0]:"Date"})
cpi_val_series = pd.DataFrame([cpi_values])
cpi_val_series = cpi_val_series.T
cpi_val_series = cpi_val_series.rename(columns={cpi_val_series.columns[0]:"CPI"})
cpi_df2 = pd.concat([date_series, cpi_val_series], axis=1).reindex(date_series.index)
cpi_df2 = cpi_df2.dropna()
cpi_df2 = cpi_df2.set_index(['Date'])
cpi_df2['CPI'] = cpi_df2['CPI'].astype(float)
cpi_df2.index = pd.to_datetime(cpi_df2.index)
cpi_df2['CPI_YoY_Pct_Chg'] = (cpi_df2['CPI'] - cpi_df2['CPI'].shift(12)) / cpi_df2['CPI'].shift(12) * 100
cpi_df2['CPI_YoY_Pct_Chg'] = cpi_df2['CPI_YoY_Pct_Chg'].round(2)
cpi_df2['CPI_New_Vehicles'] = (cpi_df2['CPI_YoY_Pct_Chg'] - cpi_df2['CPI_YoY_Pct_Chg'].mean()) / cpi_df2['CPI_YoY_Pct_Chg'].std()
cpi_df2['CPI_New_Vehicles'] = cpi_df2['CPI_New_Vehicles'].round(2)
cpi_new_vehicles_df = cpi_df2[['CPI_New_Vehicles']]
cpi_new_vehicles_df

,CPI_New_Vehicles
Date,
1947-03-01,NaN
1947-06-01,NaN
1947-09-01,NaN
1947-11-01,NaN
1947-12-01,NaN
...,...
2021-03-01,-0.16
2021-04-01,-0.04
2021-05-01,0.34


In [13]:
# Read data
# CPI data
cpi_df = pd.read_excel('./Inputs/BLS_data/CPI_for_All_Urban_Consumers_Used_Vehicles.xlsx', engine='openpyxl', header=10)
header_row = 0
cpi_df.columns = cpi_df.iloc[header_row]
cpi_df = cpi_df.drop(header_row)
cpi_df = cpi_df.reset_index(drop=True)
del cpi_df['HALF1']
del cpi_df['HALF2']
cpi_df['Year'] = cpi_df['Year'].astype(int)
months = cpi_df.columns.tolist()
months.pop(0)
years = cpi_df['Year'].to_list()

# Build year and month dataframe
date = []
for year in years:
    for month in months:
        x = (f"{year}-{month}")
        date.append(x)

del cpi_df['Year']

# Build CPI values list
cpi_values = []
x = []
for i in range(len(cpi_df.index)):
    x = cpi_df.values.tolist()[i]
    cpi_values = cpi_values + x

date_series = pd.DataFrame([date])
date_series = date_series.T
date_series = date_series.rename(columns={date_series.columns[0]:"Date"})
cpi_val_series = pd.DataFrame([cpi_values])
cpi_val_series = cpi_val_series.T
cpi_val_series = cpi_val_series.rename(columns={cpi_val_series.columns[0]:"CPI"})
cpi_df2 = pd.concat([date_series, cpi_val_series], axis=1).reindex(date_series.index)
cpi_df2 = cpi_df2.dropna()
cpi_df2 = cpi_df2.set_index(['Date'])
cpi_df2['CPI'] = cpi_df2['CPI'].astype(float)
cpi_df2.index = pd.to_datetime(cpi_df2.index)
cpi_df2['CPI_YoY_Pct_Chg'] = (cpi_df2['CPI'] - cpi_df2['CPI'].shift(12)) / cpi_df2['CPI'].shift(12) * 100
cpi_df2['CPI_YoY_Pct_Chg'] = cpi_df2['CPI_YoY_Pct_Chg'].round(2)
cpi_df2['CPI_Used_Vehicles'] = (cpi_df2['CPI_YoY_Pct_Chg'] - cpi_df2['CPI_YoY_Pct_Chg'].mean()) / cpi_df2['CPI_YoY_Pct_Chg'].std()
cpi_df2['CPI_Used_Vehicles'] = cpi_df2['CPI_Used_Vehicles'].round(2)
cpi_used_vehicles_df = cpi_df2[['CPI_Used_Vehicles']]
cpi_used_vehicles_df

,CPI_Used_Vehicles
Date,
1952-12-01,NaN
1953-01-01,NaN
1953-02-01,NaN
1953-03-01,NaN
1953-04-01,NaN
...,...
2021-03-01,0.78
2021-04-01,2.21
2021-05-01,3.29


In [19]:
merged_df1 = cpi_all_df.merge(cpi_food_df, how='outer', left_index=True, right_index=True)
merged_df2 = cpi_housing_df.merge(cpi_medical_df, how='outer', left_index=True, right_index=True)
merged_df3 = cpi_education_df.merge(cpi_fuel_df, how='outer', left_index=True, right_index=True)
merged_df4 = cpi_new_vehicles_df.merge(cpi_used_vehicles_df, how='outer', left_index=True, right_index=True)
merged_df5 = merged_df1.merge(merged_df2, how='outer', left_index=True, right_index=True)
merged_df6 = merged_df3.merge(merged_df4, how='outer', left_index=True, right_index=True)
merged_df7 = merged_df5.merge(merged_df6, how='outer', left_index=True, right_index=True)
merged_df7

,CPI_All,CPI_Food,CPI_Housing,CPI_Medical,CPI_Education,CPI_Motor_Fuel,CPI_New_Vehicles,CPI_Used_Vehicles
Date,,,,,,,,
1913-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2021-03-01,-0.12,0.01,-0.66,-1.20,-0.27,1.12,-0.16,0.78
2021-04-01,0.19,-0.14,-0.51,-1.31,-0.12,2.87,-0.04,2.21
2021-05-01,0.36,-0.17,-0.42,-1.50,0.04,3.30,0.34,3.29


In [20]:
cpi_merge_plot = merged_df7.hvplot.line(title='CPI Compiled',
                                                grid=True,height=500,width=800)
cpi_merge_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)